In [1]:
!git clone https://github.com/juanthomasw/Spatial_Transformer_Networks

Cloning into 'Spatial_Transformer_Networks'...
remote: Enumerating objects: 259, done.
remote: Counting objects: 100% (168/168), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 259 (delta 85), reused 26 (delta 11), pack-reused 91
Receiving objects: 100% (259/259), 19.04 MiB | 22.30 MiB/s, done.
Resolving deltas: 100% (103/103), done.


In [2]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Change the current directory to the desired directory
%cd "/content/drive/My Drive/SM"

# Copy the ZIP file to the local Colab environment
!cp "data_yolov7_resized.zip" "/content"

%cd "/content"

# Unzip the file
!unzip "data_yolov7_resized.zip"

# Remove the downloaded ZIP file
!rm "data_yolov7_resized.zip"

Mounted at /content/drive
/content/drive/My Drive/SM
/content
Archive:  data_yolov7_resized.zip
   creating: labels/
   creating: labels/train/
  inflating: labels/train/IMG_1.txt  
  inflating: labels/train/IMG_101.txt  
  inflating: labels/train/IMG_102.txt  
  inflating: labels/train/IMG_103.txt  
  inflating: labels/train/IMG_104.txt  
  inflating: labels/train/IMG_106.txt  
  inflating: labels/train/IMG_107.txt  
  inflating: labels/train/IMG_108.txt  
  inflating: labels/train/IMG_109.txt  
  inflating: labels/train/IMG_11.txt  
  inflating: labels/train/IMG_111.txt  
  inflating: labels/train/IMG_112.txt  
  inflating: labels/train/IMG_113.txt  
  inflating: labels/train/IMG_114.txt  
  inflating: labels/train/IMG_116.txt  
  inflating: labels/train/IMG_117.txt  
  inflating: labels/train/IMG_118.txt  
  inflating: labels/train/IMG_119.txt  
  inflating: labels/train/IMG_12.txt  
  inflating: labels/train/IMG_121.txt  
  inflating: labels/train/IMG_122.txt  
  inflating: labels/

In [3]:
# Copy the ZIP file to the local Colab environment
%cd "/content/drive/My Drive/SM"
!mkdir -p /content/detection
!cp "DetectLabels.zip" "/content/detection"

%cd "/content/detection"

# Unzip the file
!unzip "DetectLabels.zip"

# Remove the downloaded ZIP file
!rm "DetectLabels.zip"

/content/drive/My Drive/SM
/content/detection
Archive:  DetectLabels.zip
  inflating: IMG_10.txt              
  inflating: IMG_100.txt             
  inflating: IMG_1000.txt            
  inflating: IMG_105.txt             
  inflating: IMG_110.txt             
  inflating: IMG_115.txt             
  inflating: IMG_120.txt             
  inflating: IMG_125.txt             
  inflating: IMG_130.txt             
  inflating: IMG_135.txt             
  inflating: IMG_140.txt             
  inflating: IMG_145.txt             
  inflating: IMG_15.txt              
  inflating: IMG_150.txt             
  inflating: IMG_155.txt             
  inflating: IMG_160.txt             
  inflating: IMG_165.txt             
  inflating: IMG_170.txt             
  inflating: IMG_175.txt             
  inflating: IMG_180.txt             
  inflating: IMG_185.txt             
  inflating: IMG_190.txt             
  inflating: IMG_195.txt             
  inflating: IMG_20.txt              
  inflating: IM

In [30]:
import os
import sys
sys.path.append('/content/Spatial_Transformer_Networks/STN')
from STN_Test import *
import numpy as np
import argparse
import torch
import time
import cv2
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

from torchvision import transforms
import matplotlib.pyplot as plt

# Load the STNet model and weights
STN = STNet()
STN.to(device)
STN.load_state_dict(torch.load('/content/Spatial_Transformer_Networks/STN/weights/Final_STN_model.pth', map_location=lambda storage, loc: storage))
STN.eval()

# Directory where label files are located
label_directory = '/content/labels/val/'

# List all label files in the directory
label_files = [f for f in os.listdir(label_directory) if f.endswith(".txt")]

# Define a function to convert YOLO format labels to bounding box coordinates
def yolo_to_bbox(yolo_label, image_width, image_height):
    parts = yolo_label.strip().split()
    class_idx = int(parts[0])
    x_center, y_center, width, height = map(float, parts[1:5])  # Skip the confidence value

    x1 = int((x_center - width / 2) * image_width)
    y1 = int((y_center - height / 2) * image_height)
    x2 = int((x_center + width / 2) * image_width)
    y2 = int((y_center + height / 2) * image_height)

    return class_idx, x1, y1, x2, y2


# Define a directory to save the transformed images
output_directory = '/content/transformed_images/'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Process each label file and image
for label_file in label_files:
    label_path = os.path.join(label_directory, label_file)
    image_name = os.path.splitext(label_file)[0]  # Extract the image name from the label file
    image_path = os.path.join('/content/images/val', image_name + '.jpg')  # Updated image path

    # Check if the image file exists
    if not os.path.isfile(image_path):
        print(f"Image file not found: {image_path}")
        continue  # Skip this iteration

    # Load and preprocess the image
    image = cv2.imread(image_path)

    # Check if the image was loaded successfully
    if image is None:
        print(f"Failed to load image: {image_path}")
        continue  # Skip this iteration

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Read all labels from the label file
    with open(label_path, 'r') as file:
        yolo_labels = file.readlines()

    # Process each label in the label file
    for idx, yolo_label in enumerate(yolo_labels, 1):
        class_idx, x1, y1, x2, y2 = yolo_to_bbox(yolo_label, image.shape[1], image.shape[0])

        img_box = image[y1:y2+1, x1:x2+1, :]
        im = cv2.resize(img_box, (564, 144), interpolation=cv2.INTER_CUBIC)
        im = (np.transpose(np.float32(im), (2, 0, 1)) - 127.5) * 0.0078125
        data = torch.from_numpy(im).float().unsqueeze(0).to(device)  # torch.Size([1, 3, 24, 94])

        # Apply the spatial transformation
        transfer = STN(data)
        transformed_img = convert_image(transfer)

        transformed_img = cv2.resize(transformed_img, (752, 192), interpolation=cv2.INTER_CUBIC)

        # Save the transformed image to a file with a unique name
        output_filename = f"{image_name}_{idx}_transformed.jpg"
        output_path = os.path.join(output_directory, output_filename)
        cv2.imwrite(output_path, cv2.cvtColor(transformed_img, cv2.COLOR_RGB2BGR))

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4358: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4296: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


In [28]:
import shutil

folder_path = '/content/transformed_images'

# Remove the folder and its contents
shutil.rmtree(folder_path)

In [31]:
%cd /content

!zip -r STN_Transformed.zip /content/transformed_images

/content
  adding: content/transformed_images/ (stored 0%)
  adding: content/transformed_images/IMG_290_4_transformed.jpg (deflated 1%)
  adding: content/transformed_images/IMG_810_1_transformed.jpg (deflated 2%)
  adding: content/transformed_images/IMG_660_2_transformed.jpg (deflated 1%)
  adding: content/transformed_images/IMG_455_1_transformed.jpg (deflated 3%)
  adding: content/transformed_images/IMG_550_2_transformed.jpg (deflated 1%)
  adding: content/transformed_images/IMG_940_1_transformed.jpg (deflated 1%)
  adding: content/transformed_images/IMG_105_1_transformed.jpg (deflated 1%)
  adding: content/transformed_images/IMG_160_1_transformed.jpg (deflated 3%)
  adding: content/transformed_images/IMG_410_1_transformed.jpg (deflated 2%)
  adding: content/transformed_images/IMG_865_1_transformed.jpg (deflated 2%)
  adding: content/transformed_images/IMG_685_1_transformed.jpg (deflated 2%)
  adding: content/transformed_images/IMG_200_2_transformed.jpg (deflated 1%)
  adding: content